# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [1]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots
using LaTeXStrings
using JLD

In [14]:
include("../../../src/decompositions/tensors/naive_low_multilinear_rank_approximation.jl")
include("../../../src/decompositions/tensors/curvature_corrected_low_multilinear_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")
include("../../../src/utils/curvature_corrected_step_size.jl")

curvature_corrected_stepsize (generic function with 2 methods)

### Load data and construct manifold ###

In [3]:
# load data
# ni = niread("data/nifti_dt.nii.gz") * 1e9;
ni = niread("../../../data/nifti_dt_nonlinear.nii.gz") * 1e9;
size(ni)

(112, 112, 50, 1, 6)

In [4]:
 d1, d2, d3, _, d = size(ni)

# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [ # data ordered as [xx, yx, yy, zx, zy, zz]
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];
    
# pick a 2D slice 
data = predata[38:77,36:75,15];
# data = data[1:40,1:40]

# construct data manifold
n = size(data)
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M) # -> different than in 1D

6

In [5]:
# Export slice image
asymptote_export_SPD("results/Camino2D_orig.asy", data=data);

### Construct low rank approximation ###

In [6]:
unrolled_data = []
for mat in data
    push!(unrolled_data,mat)
end
q = mean(M, unrolled_data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [53]:
# Time loss evaluation
# curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
# Time gradient evaluation

In [7]:
R_q, U  = naive_low_multilinear_rank_approximation(M, q, data, [1,1])

step_size = curvature_corrected_stepsize(M, q, data, Tuple(U))
println(step_size)

0.003389115308303295


In [8]:
# TODO find a good step size...
_, __ = curvature_corrected_low_multilinear_rank_approximation(M, q, data, [1,1]; max_iter=20, change_tol=1e-4); 

0.003389115308303295
Initial  F(x): 1.48428053930 | 
# 1     change: 1.910364194 |  F(x): 1.31445108513 | 
# 2     change: 1.420834315 |  F(x): 1.21432902341 | 
# 3     change: 1.155823375 |  F(x): 1.14750428054 | 
# 4     change: 0.957476120 |  F(x): 1.10130208357 | 
# 5     change: 0.806045865 |  F(x): 1.06833340838 | 
# 6     change: 0.688534627 |  F(x): 1.04412666351 | 
# 7     change: 0.595931265 |  F(x): 1.02589103899 | 
# 8     change: 0.521894751 |  F(x): 1.01183408874 | 
# 9     change: 0.461885827 |  F(x): 1.00077410738 | 
# 10    change: 0.412598487 |  F(x): 0.99191335552 | 
# 11    change: 0.371587914 |  F(x): 0.98470149774 | 
# 12    change: 0.337024226 |  F(x): 0.97875111923 | 
# 13    change: 0.307525829 |  F(x): 0.97378418256 | 
# 14    change: 0.282043365 |  F(x): 0.96959737104 | 
# 15    change: 0.259776469 |  F(x): 0.96603924662 | 
# 16    change: 0.240112566 |  F(x): 0.96299495655 | 
# 17    change: 0.222581100 |  F(x): 0.96037585106 | 
# 18    change: 0.206818968 |

In [9]:
max_iter = 25
rank = 10

nRr_q, nUr = naive_low_multilinear_rank_approximation(M, q, data, [rank,rank]); 
ccRr_q, ccUr = curvature_corrected_low_multilinear_rank_approximation(M, q, data, [rank,rank]; max_iter=max_iter, change_tol=1e-4); 

save("results/Camino2D/jld/RqU-size$(n[1])x$(n[2])-iters$(max_iter)-rank$(rank).jld", "nRr_q", nRr_q, "nUr", nUr, "ccRr_q", ccRr_q, "ccUr", ccUr)

0.00046437898778844
Initial  F(x): 2.95612281412 | 
# 1     change: 1.266515237 |  F(x): 2.39905047296 | 
# 2     change: 0.950700203 |  F(x): 2.07483688409 | 
# 3     change: 0.765592139 |  F(x): 1.85928822555 | 
# 4     change: 0.648259839 |  F(x): 1.70218577379 | 
# 5     change: 0.566651257 |  F(x): 1.58091652204 | 
# 6     change: 0.505069269 |  F(x): 1.48395010097 | 
# 7     change: 0.455779717 |  F(x): 1.40464234510 | 
# 8     change: 0.414785199 |  F(x): 1.33875001411 | 
# 9     change: 0.379857575 |  F(x): 1.28334724837 | 
# 10    change: 0.349642395 |  F(x): 1.23630606189 | 
# 11    change: 0.323244327 |  F(x): 1.19602167923 | 
# 12    change: 0.300027115 |  F(x): 1.16125349581 | 
# 13    change: 0.279511599 |  F(x): 1.13102555324 | 
# 14    change: 0.261320510 |  F(x): 1.10456025535 | 
# 15    change: 0.245146764 |  F(x): 1.08123204804 | 
# 16    change: 0.230734204 |  F(x): 1.06053384961 | 
# 17    change: 0.217865301 |  F(x): 1.04205204485 | 
# 18    change: 0.206352923 | 

In [10]:
# retrieve results
rank_range = [1, 2, 5]
# rank_range = [1, 5, 10, 15, 20]

nR_q = []
nU = []
ccR_q = []
ccU = []
for rank in rank_range
    dic = load("results/Camino2D/jld/RqU-size$(n[1])x$(n[2])-iters$(max_iter)-rank$(rank).jld")

    push!(nR_q, dic["nRr_q"])
    push!(nU, dic["nUr"])
    push!(ccR_q, dic["ccRr_q"])
    push!(ccU, dic["ccUr"])
end

In [11]:
num_ranks = length(rank_range)
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(num_ranks)
predicted_naive_distances_r= zeros(num_ranks)
true_naive_distances_r= zeros(num_ranks)

curvature_corrected_tangent_distances_r = zeros(num_ranks)
predicted_curvature_corrected_distances_r = zeros(num_ranks)
true_curvature_corrected_distances_r = zeros(num_ranks)

for (idx,rank) in enumerate(rank_range)
    naive_log_q_data_r = Symmetric.([sum([nR_q[idx][i,j] * nU[idx][1][k,i] * nU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[idx][i,j] * ccU[idx][1][k,i] * ccU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)

    # compute relative tangent space error
    naive_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance

    # compute relative manifold error
    predicted_naive_distances_r[idx] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[idx] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[idx] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[idx] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
end

In [12]:
plot(rank_range, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(0,1), xlims=(1,max(rank_range...)),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/Camino2D_errors_by_rank.png")
plot(rank_range, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(1e-16,1), xlims=(1,max(rank_range...)), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_logerrors_by_rank.png"

In [13]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(rank_range, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(rank_range...)),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/Camino2D_discrepancy_by_rank.png")
plot(rank_range, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(rank_range...)), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_logdiscrepancy_by_rank.png"